# Folium Project

## Description

### Installation Packages

In [705]:
%pip install pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [706]:
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [707]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [708]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [709]:
%pip install jenkspy

Note: you may need to restart the kernel to use updated packages.


In [710]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pyodbc
import pandas as pd
import binascii
import json
import folium
from folium.plugins import HeatMap
import requests
import geopandas as gpd
import  numpy as np
from branca.colormap import linear
import branca.colormap as cm
from folium.plugins import MiniMap
import geopy.distance
from itertools import combinations

In [29]:
# Définition ses informations de connexion
# Aller dans SSMIS -> clique droit sur AdventureWorks2022 -> Properties et récupérer le nom du server et de la database
server = 'LAPTOP-SQ0G8B4V\SQLEXPRESS'
database = 'AdventureWorks2022'

# Établir la connexion
conn = pyodbc.connect(
    f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Windows Authentication"
)


### Code Découverte Folium

#### Preparation des données 

In [30]:
# Exécutation des requêtes SQL et traitez les résultats avec pandas
query = "SELECT V.[Name], A.AddressID, A.AddressLine1, A.AddressLine2, A.City, A.PostalCode, A.SpatialLocation.Lat AS Latitude ,A.SpatialLocation.Long AS Longitude FROM Purchasing.Vendor V INNER JOIN Person.BusinessEntityAddress BEA ON V.BusinessEntityID = BEA.BusinessEntityID INNER JOIN Person.Address A ON BEA.AddressID = A.AddressID"
data = pd.read_sql(query, conn)


C:\Users\veyss\AppData\Local\Temp\ipykernel_64840\3958794958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


In [31]:
data

,Name,AddressID,AddressLine1,AddressLine2,City,PostalCode,Latitude,Longitude
0,Bergeron Off-Roads,311,9830 May Way,None,Mill Valley,59715,45.715117,-110.930516
1,Hill's Bicycle Service,348,3664 Colt Ct.,None,Richmond,94801,37.926440,-122.196945
2,Circuit Cycles,322,1 Mt. Dell Drive,None,Portland,97205,45.516432,-122.846246
3,Green Lake Bike Company,344,2342 Peachwillow,None,Denver,80203,39.749846,-105.108452
4,Consumer Cycles,327,3 Chablis Court,None,Torrance,90505,33.847532,-118.268363
...,...,...,...,...,...,...,...,...
99,Bloomington Multisport,314,218 Fall Creek Road,None,West Covina,91791,34.048017,-118.069866
100,Carlson Specialties,318,2313 B Southampton Rd,None,Missoula,59801,46.790615,-113.976004
101,"Compete, Inc.",325,9491 Toyon Dr,None,Dallas,75201,32.786195,-96.926752
102,Chicago City Saddles,320,3 Gehringer Drive,None,Daly City,94015,37.634691,-122.581953


#### Test découverte Folium 

In [32]:
# Création d'une carte Folium 
m = folium.Map(location = [40, -95], zoom_start = 4)
display(m)

In [33]:
# Ajout des marqueurs à la carte
for index, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row[['Name','City']],
        icon=folium.Icon(color='red'),

    ).add_to(m)

# Affichez la carte 
m

In [34]:
# Création d'une liste de points de données pour la HeatMap
heat_data = [[row['Latitude'], row['Longitude']] for index, row in data.iterrows()]

# Création de la HeatMap et ajout à la carte Folium
HeatMap(heat_data).add_to(m)

m

In [57]:
m3 = folium.Map(location=[22.5, -115], zoom_start=4)

video = folium.raster_layers.VideoOverlay(
    video_url="https://www.mapbox.com/bites/00188/patricia_nasa.webm",
    bounds=[[32, -130], [13, -100]],
    opacity=0.65,
    attr="Video from patricia_nasa",
    autoplay=True,
    loop=False,
)

video.add_to(m3)

m3.save("folium_video.html")

### Mise en Situation 2

 Nous sommes en 2012, le VP des ventes a remarqué lors du dernier rapport que vous lui avez fournit sur l'année 2012, que les Etats-Unis sont la plus grosse source de vente de l'entreprise. Le pays étant constitué d'un grand nombre de province, il aimerait pouvoir visualiser les provinces et les villes selon le total des sous totaux avec un échelle de couleur pour les provinces et à l'aide de cercle pour les villes dont la taille dépend du total. De plus il aimerait visualiser les chiffres exact pour chaque province et ville facilement.

Inconvénient de folium :
Il faut a chaque fois recreer la carte pour ajouter les modifs

In [35]:
query2 = "SELECT ST.StateProvinceCode AS STCODE, ST.[Name] AS Province, SUM(SOH.SubTotal) AS TotalSales FROM Sales.SalesOrderHeader SOH INNER JOIN  Person.[Address] A ON SOH.BillToAddressID = A.AddressID INNER JOIN Person.StateProvince ST ON A.StateProvinceID = ST.StateProvinceID INNER JOIN Person.CountryRegion CR ON ST.CountryRegionCode = CR.CountryRegionCode WHERE CR.[Name] = 'United States' AND DATEPART(year,SOH.OrderDate)=2012 GROUP BY ST.[Name], ST.StateProvinceCode ORDER BY 1 DESC"

In [36]:
query3 = "SELECT A.City, SUM(SOH.SubTotal) AS TotalSales, A.SpatialLocation.Lat AS Latitude ,A.SpatialLocation.Long AS Longitude FROM Sales.SalesOrderHeader SOH INNER JOIN  Person.[Address] A ON SOH.BillToAddressID = A.AddressID INNER JOIN Person.StateProvince ST ON A.StateProvinceID = ST.StateProvinceID INNER JOIN Person.CountryRegion CR ON ST.CountryRegionCode = CR.CountryRegionCode WHERE CR.[Name] = 'United States' AND DATEPART(year,SOH.OrderDate)=2012 GROUP BY A.City, A.SpatialLocation.Long, A.SpatialLocation.Lat ORDER BY SUM(SOH.SubTotal) DESC"

In [37]:
data_exo2 = pd.read_sql(query2, conn)
data_exo22 = pd.read_sql(query3, conn)

C:\Users\veyss\AppData\Local\Temp\ipykernel_64840\3610141962.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_exo2 = pd.read_sql(query2, conn)
C:\Users\veyss\AppData\Local\Temp\ipykernel_64840\3610141962.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_exo22 = pd.read_sql(query3, conn)


In [38]:
data_exo22

,City,TotalSales,Latitude,Longitude
0,Carson,407433.5041,33.807285,-118.267754
1,San Antonio,375493.4641,29.470559,-98.429057
2,Houston,327015.6370,29.803783,-95.505601
3,Memphis,317507.3898,35.027174,-89.754655
4,Tooele,313824.7287,40.563292,-112.253172
...,...,...,...,...
1155,Cedar Park,159.5580,30.460629,-97.831450
1156,Puyallup,126.8942,47.211762,-122.141962
1157,Eureka,91.1760,40.841938,-124.029302
1158,Riverton,28.8404,40.493820,-112.050659


In [39]:
# Création d'une carte Folium 
m2 = folium.Map(location=[48, -102], zoom_start=3)

MiniMap(toggle_display=True).add_to(m2)

In [40]:
#Importation du découpage des provinces des Etats-Unis
us_states = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()

In [41]:
us_states_gdf = gpd.read_file("https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json")

In [42]:
data_exo2['STCODE'] = data_exo2['STCODE'].str.strip()

existing_states = data_exo2['STCODE'].tolist()

new_state_data = []
for _, row1 in us_states_gdf.iterrows():
    if row1['id'] not in existing_states:
        new_state_df = pd.DataFrame({
            'STCODE': [row1['id']],
            'Province': [row1['name']],
            'TotalSales': [0]
        })
        
        new_state_data.append(new_state_df)

In [43]:
data_exo2 = pd.concat([data_exo2] + new_state_data, ignore_index=True)

In [44]:
data_exo2['TotalSales_log'] = np.log1p(data_exo2['TotalSales'])

In [45]:
# Calcul des coordonnées centrales pour chaque ville

def get_centroids(coords):
    centroid = (sum(coords[:, 0]) / len(coords), sum(coords[:, 1]) / len(coords))
    return pd.Series({'Latitude': centroid[0], 'Longitude': centroid[1]})

# # Calcul des coordonnées centrales pour chaque ville
Coords = data_exo22.groupby('City')[['Latitude', 'Longitude']].apply(lambda x: get_centroids(x.values)).reset_index()

In [47]:
cp = folium.Choropleth(
    geo_data=us_states,
    data=data_exo2,
    columns=["STCODE","TotalSales_log"],
    key_on="feature.id",
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Sales (Log Scale)",
    tooltip="State: {STCODE}, Total Sales: {TotalSales}",
    bins =15
)
cp.add_to(m2)

In [48]:
# Création de cercles pour chaque ville avec les ventes correspondantes
for index, row in Coords.iterrows():
    
    TotalSales_City = data_exo22.loc[data_exo22['City'] == row['City'], 'TotalSales'].sum()
    TotalSales_Cityd = TotalSales_log = np.log1p(TotalSales_City)
    TotalSales_Cityd = "${:,.2f}".format(TotalSales_City)
 
    CM = folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=TotalSales_log,  # Ajustez la taille des cercles selon les ventes
        fill=False,
        fill_color='Blue',
        fill_opacity=0.4,
        color=None,
        popup=folium.Popup(f"City :{row['City']},<br/> Ventes : {TotalSales_Cityd}", max_width=300)
    ).add_to(m2)

In [49]:
#Ajoute le $
data_exo2['TotalSales'] = data_exo2['TotalSales'].apply(lambda x: "${:,.2f}".format(x))

In [51]:
state_data_indexed = data_exo2.set_index('STCODE')
  
for s in cp.geojson.data['features']:
    s['properties']['TotalSales'] = state_data_indexed.loc[s['id'], 'TotalSales']
  
  # ajout du Tooltip au choropleth
folium.GeoJsonTooltip(['name', 'TotalSales']).add_to(cp.geojson)
  
folium.LayerControl().add_to(m2)

In [52]:
m2

In [53]:
m2.save("folium_exo2.html")

#### Fermeture de la connection avec AdventureWorks2022

In [54]:
#Fermeture de la connexion
conn.close()